# Coursework Start 
This is my coursework wow how amazing that is increible

Insatlls: 
!pip install scikit-learn
!pip install nltk
!pip install -q wordcloud
!pip install gensim


In [3]:
# Setup
import string
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
import numpy 

# for Lemmatizer
from nltk.stem import WordNetLemmatizer
import wordcloud
import nltk
nltk.download('wordnet')
# defines  lemmatizer for use
lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


# Task 1 - Bag of Words with tf*idf

Current best accuracy from validation dataset provided is 58.88%

In [4]:

# basic Tokenisation function, removes all puntuation apart from - and splits on spaces whilst replacing uppercase with lower 
def tokenizeText(text):
  # Remove punctuation except apostrophes
  translator = str.maketrans('', '', (string.punctuation.replace("-", "")))
  text = text.translate(translator).lower()

  # Splits on spaces 
  tokens = text.split()

  # lemmatizes each token 
  for i in range(0, len(tokens)):
    tokens[i] = lemmatizer.lemmatize(tokens[i])

  return tokens


def getDocTermVecs(corpus):
  # uses sklearn function to transform a corpus into a document-term matrix whilst also tokenizing using my tokenizeText function
  vectorizer = TfidfVectorizer(tokenizer=tokenizeText) 
  X = vectorizer.fit_transform(corpus)
  
  #gets relevent terms in correct order from doc-term matrix 
  terms = vectorizer.get_feature_names_out()

  # returning in form terms, vecor representations
  return terms.tolist(), X.transpose().toarray()

# returns corpus, corpusTokenized
def extractCorpus(corpusDest):
  # gets all descriptions from training set 
  corpus = []
  # corpusTokenized used for word2vec later
  corpusTokenized = []
  labels = []
  with open(corpusDest, newline='') as trainingData:
    for row in csv.reader(trainingData):
      if row[3] == 'comedy': continue
      tokenizedScen = tokenizeText(row[2])
      corpusTokenized.append(tokenizedScen)
      corpus.append(row[2])
      labels.append([int(row[3]),int(row[4]),int(row[5]),int(row[6]),
                     int(row[7]),int(row[8]),int(row[9]),int(row[10]),int(row[11])])
      # labels.append(1)
  
  return corpus, corpusTokenized, labels

corpus, corpusTokenized, labels = extractCorpus('./data/Training-dataset.csv')
# generates docTermMatrix from corpus 
terms, vecMatrix = getDocTermVecs(corpus)

  


/Users/willmoran/Documents/UNI/NLP_coursework/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [5]:

# gets the average feature vector 
avgVect = numpy.mean(vecMatrix, axis=0)

# calculates the cosine similarity between two words for task 1 bag of words
def cosineSimilarityBoW(term1, term2):
  
  # gets each term vector from the vector matrix, if a term is not in the matrix it is given the average vector matrix
  # (this seemed to improve performace by 1-3% accuracy)
  if (term1 in terms) and (term2 in terms):
    term1Vec = vecMatrix[terms.index(term1)]
    term2Vec = vecMatrix[terms.index(term2)]
  elif (term1 in terms) and not(term2 in terms):
    term1Vec = vecMatrix[terms.index(term1)]
    term2Vec = avgVect
  elif not(term1 in terms) and (term2 in terms): 
    term1Vec = avgVect
    term2Vec = vecMatrix[terms.index(term1)]
  else:
    term1Vec, term2Vec = avgVect
  

  result = 1 - spatial.distance.cosine(term1Vec, term2Vec)
  return result

def generateTask1Results(cosineSimFunction):
  # reads terms from validation dataset -> calculates results and stores results in results array
  results = []
  with open('./data/Task-1-validation-dataset.csv', newline='') as validationDataset:
    for row in csv.reader(validationDataset):
      # calculates the cosine similarity for the lematized version of each pair of words in the validation dataset
      cosineSim = cosineSimFunction(
        lemmatizer.lemmatize(row[1]), 
        lemmatizer.lemmatize(row[2]))
      
      # adds results with corrosponding pairID to results array
      resultsRow = [row[0],cosineSim]
      results.append(resultsRow)

  # writes the results to results csv
  with open('./data/Task-1-validation-dataset-results.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(results)  

generateTask1Results(cosineSimilarityBoW)


# Task 1.2 - Create word-vector respresentation (Word2Vec)

In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpusTokenized, min_count = 1, vector_size = 500,
                                             window = 7, sg = 1)



In [7]:
def cosineSimilarityW2V(term1, term2):
  try: 
    return model.wv.similarity(term1, term2)
  except:
    return 0.01

print(model.wv.get_vector('room').shape)

generateTask1Results(cosineSimilarityW2V)
# print(model.wv.similarity())

(500,)


# Task2.1 Classification with SVC

In [8]:
print(corpusTokenized[2])

['lankester', 'merrin', 'is', 'a', 'veteran', 'catholic', 'priest', 'and', 'exorcist', 'who', 'is', 'on', 'an', 'archaeological', 'dig', 'in', 'iraq', 'there', 'he', 'find', 'an', 'amulet', 'that', 'resembles', 'the', 'statue', 'of', 'pazuzu', 'a', 'demon', 'whom', 'merrin', 'had', 'defeated', 'year', 'before', 'merrin', 'then', 'realizes', 'the', 'demon', 'ha', 'returned', 'to', 'seek', 'revenge', 'meanwhile', 'in', 'georgetown', 'actress', 'chris', 'macneil', 'is', 'living', 'on', 'location', 'with', 'her', '12-year-old', 'daughter', 'regan', 'where', 'chris', 'ha', 'just', 'wrapped', 'the', 'final', 'scene', 'of', 'a', 'film', 'about', 'student', 'activism', 'directed', 'by', 'her', 'friend', 'and', 'associate', 'burke', 'dennings', 'after', 'playing', 'with', 'a', 'ouija', 'board', 'and', 'contacting', 'a', 'supposedly', 'imaginary', 'friend', 'whom', 'she', 'call', 'captain', 'howdy', 'regan', 'begin', 'acting', 'strangely', 'including', 'making', 'mysterious', 'noise', 'stealing'

In [9]:
import tensorflow as tf

# VOCAB_SIZE = 1000

encoder = tf.keras.layers.TextVectorization()
encoder.adapt(corpus)
vocab = numpy.array(encoder.get_vocabulary())





In [10]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(350)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    # tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(9, activation='softmax')
])

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.legacy.Adam(1e-4),
              metrics=['accuracy'])

# labelsVectorised = numpy.asarray(labels).astype('float32').reshape((-1,1))
history = model.fit(corpus, labels, epochs=5,
                    validation_steps=30)

Epoch 1/5
  6/259 [..............................] - ETA: 22:16 - loss: 4.3587 - accuracy: 0.0781

In [ ]:
prediction = model.predict(['''In the crime-ridden city of Tremont, renowned investigative journalist, Amelia Williams, wages war against an influential criminal syndicate. The Syndicate, led by the ruthless mafia boss, Victor Russo, smuggles drugs and extorts businesses, instilling fear throughout the entire city. Amidst the chaos, Amelia's investigative reporting catches the attention of Jared Bennett, a former Special Forces operative haunted by his personal demons.

Amelia, driven by her desire for justice, covertly gathers vital evidence against the Syndicate, documenting their illegal activities in a clandestine exposé. However, her secret investigation is abruptly cut short when Amelia is found brutally murdered in her apartment. The loss devastates her loyal intern, Julia Evans, who becomes determined to avenge her mentor's death and complete the mission Amelia started.

Upon hearing the tragic news, Jared, who had developed feelings for Amelia, is consumed by a thirst for revenge. Haunted by his past failures, Jared pledges to help Julia expose the Syndicate and bring justice to Amelia's killers. With their hearts set on vengeance, they form an unlikely alliance, setting in motion a thrilling tale of violence, romance, and redemption.

As Jared and Julia delve deeper into the treacherous underworld of Tremont, they discover an unexpected ally in the form of Anthony Marino, an undercover cop who has infiltrated the Syndicate. Together, the trio infiltrates a high-profile underground casino run by the Syndicate, hoping to find evidence to dismantle Russo's empire. Amidst the chaos of the casino, Jared crosses paths with Victoria Russo, Victor's estranged daughter. Their connection is immediate, their attraction undeniable, but Victoria is torn between her loyalty to her father and the sense of justice that Jared represents.

As the investigation intensifies, the Syndicate grows suspicious, and Victor Russo takes drastic measures to protect his criminal empire. He assigns his merciless enforcer, Dimitri Saxon, to track down and eliminate the threats. Saxon, a lethal hitman with his own dark past, is relentless in his pursuit of Jared, Julia, and Anthony, leaving a path of violence in his wake.

In a climactic battle, the trio confronts the Syndicate at an abandoned warehouse, pitting their wits, skills, and firepower against Russo's formidable criminal organization. Victoria makes a fateful decision, siding with Jared and Julia, igniting a passionate battle for redemption, love, and justice.

Amidst the explosive conflict, long-buried secrets are unearthed, shedding light on the tragic events that brought Jared, Amelia, and Victoria to this final confrontation. The truth becomes a vengeful weapon, empowering our heroes to reveal the Syndicate's darkest secrets and dismantle its oppressive grip on Tremont.

In an emotional and action-packed finale, Jared confronts Victor Russo, facing not only the man responsible for Amelia's death but also his own guilt over past failures. The battle ends with the triumph of justice, as Russo meets his demise, and Julia finally avenges her beloved mentor.

With the Syndicate dismantled, Tremont begins to heal, and the remaining collaborators face the consequences of their actions. As the dust settles, Jared and Victoria, having survived the storm, find solace and love in each other's arms. Together, they commit to rebuilding their lives, forever changed by their shared journey of heartbreak, revenge, and ultimately, redemption.
'''])
print(prediction)


1/1 [==============================] - 1s 860ms/step
[[0.05772711 0.08939094 0.10727655 0.01113144 0.3150721  0.08923353
  0.11422064 0.00992511 0.20602252]]
